In [ ]:
import os
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
)
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.systems.all import Simulator
from airo_drake.stations.station_utils import AddPackagePaths
from airo_drake.downloaders.google_scanned_objects import get_google_scanned_object


In [ ]:
meshcat = StartMeshcat()

In [ ]:
def GetModelDirective(model_name):
    model_directory = get_google_scanned_object(model_name)
    model_sdf = os.path.join(model_directory, "model.sdf")

    model_directive = f"""- add_model:
            name: {model_name}
            file: file://{model_sdf}
    """
    return model_directive

In [ ]:
cube_name = "Granimals_20_Wooden_ABC_Blocks_Wagon_85VdSftGsLi"
cube_directive = GetModelDirective(cube_name)

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
AddPackagePaths(parser)

model_directives = "directives:\n" + cube_directive

directives = LoadModelDirectivesFromString(model_directives)
ProcessModelDirectives(directives, parser)
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat)
diagram = builder.Build()

In [ ]:
simulator = Simulator(diagram)
visualizer.StartRecording(False)
simulator.AdvanceTo(0.05)
visualizer.PublishRecording()

### TODO clean up below

In [ ]:
brick_directive = """- add_model:
        name: brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
"""
planner = GraspPlanner(meshcat)
SimulateGrasp(brick_directive, planner, meshcat)

In [ ]:
from pydrake.common import FindResourceOrThrow, temp_directory
import os
temp_dir = temp_directory()

# Create a simple cylinder model.
cylinder_sdf_file = os.path.join(temp_dir, "cylinder.sdf")
cylinder_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="cylinder">
    <pose>0 0 0.15 0 0 0</pose>
    <link name="cylinder_link">
      <inertial>
        <mass>0.5</mass>
        <inertia>
          <ixx>0.005833</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>0.005833</iyy>
          <iyz>0.0</iyz>
          <izz>0.005</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <cylinder>
            <radius>0.025</radius>
            <length>0.1</length>
          </cylinder>
        </geometry>
      </collision>
      <visual name="visual">
        <geometry>
          <cylinder>
            <radius>0.025</radius>
            <length>0.1</length>
          </cylinder>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 1.0</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>

"""

with open(cylinder_sdf_file, "w") as f:
    f.write(cylinder_sdf)

cylinder_directive = f"""- add_model:
        name: cylinder
        file: file://{cylinder_sdf_file}
"""

planner = GraspPlanner(meshcat)
SimulateGrasp(cylinder_directive, planner, meshcat)